In [20]:
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np 
import joblib
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from PIL import Image
import sys
from tensorflow.keras.models import model_from_json
import tensorflow_hub as hub
import pandas as pd 

In [31]:
def image_prediction(prediction_score):
    s = ""
    for i, label in enumerate(features_str):
        pred = prediction_score[i][0][0]
        s += f"{label}: predicted {1 if pred > 0.5 else 0} ({format(pred, '.4f')})"
        s +="\n"
    return s 

def display_image(image,s):
    plt.imshow(image)
    plt.title(s)
    plt.figure()
    
def open_images(inference_folder: str) -> np.ndarray:
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  
        with Image.open(img_location) as img:  
            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  
        images.append(img)
    images_array = np.vstack(images) 
    return images_array

def resize(path, dirs, resized_path):
    for item in dirs:
        print(item)
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            im = im.convert('RGB')
            f, e = os.path.splitext(path+item)
            imResize = im.resize((220,220), Image.ANTIALIAS)
            imResize.save(resized_path + item + '_resized.jpg', 'JPEG', quality=90)


In [32]:
features_str = ['Mouth_Slightly_Open','Smiling','Wearing_Lipstick','High_Cheekbones','Male','Heavy_Makeup','Wavy_Hair','Oval_Face','Pointy_Nose','Arched_Eyebrows','Big_Lips','Black_Hair''Big_Nose','Young','Straight_Hair','Brown_Hair','Bags_Under_Eyes','Wearing_Earrings','No_Beard','Bangs','Blond_Hair','Chubby','Bald']

path = "./upload_image/"
dirs = os.listdir( path )
resized_path = "./uploaded_image_resized/"
resize(path, dirs, resized_path)

321747370_1579985539139773_9125491476305201953_n.jpg


In [33]:
with open('model.json', 'r') as f: 
    json = f.read() 
model = model_from_json(json, custom_objects={'KerasLayer': hub.KerasLayer})

In [56]:
images = open_images(resized_path)
c=0
prediction_vector = []
for image in images:
    img_batch = np.expand_dims(image, axis=0)
    normalizedData = (img_batch-np.min(img_batch))/(np.max(img_batch)-np.min(img_batch))
    prediction_score= model.predict(normalizedData)
    
    for i, label in enumerate(features_str):
        pred = 1 if prediction_score[i][0][0] > 0.5 else 0 
        prediction_vector.append((label,pred))
prediction_vector       
    

1/1 [==============================] - 0s 113ms/step


[('Mouth_Slightly_Open', 0),
 ('Smiling', 0),
 ('Wearing_Lipstick', 1),
 ('High_Cheekbones', 1),
 ('Male', 1),
 ('Heavy_Makeup', 1),
 ('Wavy_Hair', 1),
 ('Oval_Face', 1),
 ('Pointy_Nose', 1),
 ('Arched_Eyebrows', 0),
 ('Big_Lips', 0),
 ('Black_HairBig_Nose', 1),
 ('Young', 0),
 ('Straight_Hair', 1),
 ('Brown_Hair', 0),
 ('Bags_Under_Eyes', 1),
 ('Wearing_Earrings', 0),
 ('No_Beard', 1),
 ('Bangs', 1),
 ('Blond_Hair', 1),
 ('Chubby', 1),
 ('Bald', 1)]

In [61]:
vector_file = open("vector.txt", "w")
string_vector = str(prediction_vector)
vector_file.write(string_vector)

433